# Analyse der Photovoltaik-Anlagen in der Schweiz
Dieses Notebook untersucht die Entwicklung und Verteilung von Photovoltaik-Anlagen (PV-Anlagen) in der Schweiz anhand aktueller Daten.

## Datenimport und Aufbereitung
In diesem Abschnitt werden die Rohdaten geladen, aufbereitet und für die weitere Analyse vorbereitet.

In [ ]:
import pandas as pd

# Einlesen der Anlagen-Daten
file_path = '../data/electricity_production_plants/ElectricityProductionPlant.csv'
df = pd.read_csv(file_path)

# Kataloge für Kategorien laden
main_categories = pd.read_csv('../data/electricity_production_plants/MainCategoryCatalogue.csv')
sub_categories = pd.read_csv('../data/electricity_production_plants/SubCategoryCatalogue.csv')
plant_categories = pd.read_csv('../data/electricity_production_plants/PlantCategoryCatalogue.csv')

# Haupt-, Unter- und Anlagenkategorien übersetzen
df['MainCategory'] = df['MainCategory'].replace(dict(zip(main_categories['Catalogue_id'], main_categories['de'])))
df['SubCategory'] = df['SubCategory'].replace(dict(zip(sub_categories['Catalogue_id'], sub_categories['de'])))
df['PlantCategory'] = df['PlantCategory'].replace(dict(zip(plant_categories['Catalogue_id'], plant_categories['de'])))

# Nur Photovoltaik-Anlagen auswählen
df = df[df['SubCategory'] == 'Photovoltaik'].copy()

# Jahr der Inbetriebnahme extrahieren
df['Year'] = pd.to_datetime(df['BeginningOfOperation'], errors='coerce').dt.year

# Auf relevante Jahre (2015–2024) filtern
df = df[df['Year'].between(2015, 2024)].copy()

# Vorschau auf die ersten Zeilen
df.head()

## Standorte aller PV-Anlagen in der Schweiz (bis 2024)
Die folgende Karte zeigt die geografische Verteilung aller Photovoltaik-Anlagen in der Schweiz bis zum Jahr 2024.

In [ ]:
# Bibliotheken für die Kartenerstellung importieren
import folium
from folium.plugins import FastMarkerCluster
from pyproj import Transformer

# Daten für Installationen bis 2024 filtern
df['BeginningOfOperation'] = pd.to_datetime(df['BeginningOfOperation'], errors='coerce')
df = df[df['BeginningOfOperation'].dt.year <= 2024]

# Karte erstellen
map = folium.Map(location=[46.8, 8.33], zoom_start=7, tiles='OpenStreetMap')

# Marker-Cluster hinzufügen
transformer = Transformer.from_crs("EPSG:2056", "EPSG:4326", always_xy=True)
coordinates = [
    [transformer.transform(row['_x'], row['_y'])[1], transformer.transform(row['_x'], row['_y'])[0]]
    for _, row in df.iterrows() if not pd.isna(row['_x']) and not pd.isna(row['_y'])
]
FastMarkerCluster(coordinates).add_to(map)

# Titel hinzufügen
title_html = '''
<div style="font-size:20px;position: absolute;z-index: 1000;left: 25%;"><b>PV-Anlagen in der Schweiz (bis 2024)</b></div>
'''
map.get_root().html.add_child(folium.Element(title_html))

# Karte speichern und anzeigen
map.save('../docs/assets/diagramme/elcoms_map.html')
map

## Kumulative Entwicklung der PV-Gesamtkapazität pro Kanton
Das folgende Liniendiagramm zeigt die Entwicklung der kumulierten Gesamtkapazität von PV-Anlagen in den Schweizer Kantonen von 2015 bis 2024.

In [ ]:
import plotly.express as px

# Kumulative Gesamtkapazität pro Kanton und Jahr berechnen
cumulative_data = (
    df.groupby(['Year', 'Canton'])['TotalPower']
    .sum()
    .groupby(level=1).cumsum()
    .reset_index()
)

# Neue Spalte für die kumulierte Leistung
cumulative_data['CumulativePower'] = cumulative_data['TotalPower']

# Top 3 Kantone im Jahr 2024 bestimmen
top_3_2024 = (
    cumulative_data[cumulative_data['Year'] == 2024]
    .sort_values(by='CumulativePower', ascending=False)
    .head(3)['Canton']
    .tolist()
)
# AG immer hervorheben
top_4_2024 = top_3_2024 + ['AG']

# Farbzuordnung für die Top-Kantone
top_colors = {
    top_4_2024[0]: 'rgb(0, 102, 204)',    # Blau
    top_4_2024[1]: 'rgb(0, 153, 76)',     # Grün
    top_4_2024[2]: 'rgb(255, 153, 51)',   # Orange
    top_4_2024[3]: 'rgb(204, 0, 0)'       # Rot
}
color_discrete_map = top_colors

# Liniendiagramm erstellen
fig = px.line(
    cumulative_data,
    x='Year',
    y='CumulativePower',
    color='Canton',
    title='Kumulative Entwicklung der PV-Gesamtkapazität pro Kanton (MW)',
    labels={'CumulativePower': 'Kumulative Gesamtkapazität (MW)', 'Canton': 'Kanton'},
    color_discrete_map=color_discrete_map
)
fig.update_traces(line=dict(width=2))

# Alle anderen Kantone grau einfärben
fig.for_each_trace(
    lambda trace: trace.update(line_color='rgb(169,169,169)') if trace.name not in top_4_2024 else None
)

fig.update_layout(
    xaxis=dict(tickmode='linear', tick0=2015, dtick=1, title=''),
    template='simple_white',
    showlegend=False
)

# Y-Offsets für die Label-Positionierung
y_offsets = [0, -10000, -20000, -30000]
vd_extra_offset = 15000  # VD etwas höher platzieren

# Labels für die Top-Kantone am rechten Rand
for i, canton in enumerate(top_4_2024):
    canton_data = cumulative_data[cumulative_data['Canton'] == canton]
    if not canton_data.empty:
        last_year = 2024
        last_value = canton_data[canton_data['Year'] == last_year]['CumulativePower'].values[0]
        y_pos = last_value + y_offsets[i]
        if canton == 'VD':
            y_pos += vd_extra_offset
        fig.add_annotation(
            x=last_year + 0.1,
            y=y_pos,
            text=canton,
            showarrow=False,
            font=dict(size=10),
            xanchor='left',
            yanchor='middle'
        )

fig.show()
fig.write_html("../docs/assets/diagramme/elcom_entwicklung_total_power.html")

## Kumulierte PV-Gesamtkapazität pro Kanton (2015–2024)
Das folgende Balkendiagramm zeigt die Entwicklung der kumulierten Gesamtkapazität von PV-Anlagen pro Kanton für die Jahre 2015 bis 2024.

In [ ]:
# Auf relevante Jahre beschränken
df = df[df['Year'].between(2015, 2024)]

# Kumulative Gesamtkapazität pro Kanton und Jahr berechnen
df_cumulative = (
    df.groupby(['Year', 'Canton'])['TotalPower']
    .sum()
    .groupby(level=1).cumsum()
    .reset_index()
)

# Funktion zur Farbzuweisung für die Top 4 Kantone pro Jahr
def assign_color(row, top4):
    if row['Canton'] == top4[0]:
        return 'rgb(0, 102, 204)'  # Blau
    elif row['Canton'] == top4[1]:
        return 'rgb(0, 153, 76)'   # Grün
    elif row['Canton'] == top4[2]:
        return 'rgb(255, 153, 51)' # Orange
    elif row['Canton'] == top4[3]:
        return 'rgb(204, 0, 0)'    # Rot
    else:
        return 'rgb(169, 169, 169)' # Grau

# Für jedes Jahr die Top 4 Kantone bestimmen und Farbe zuweisen
colored_data = []
for year in range(2015, 2025):
    year_data = df_cumulative[df_cumulative['Year'] == year].copy()
    top4 = year_data.sort_values(by='TotalPower', ascending=False).head(4)['Canton'].tolist()
    year_data['Color'] = year_data.apply(lambda row: assign_color(row, top4), axis=1)
    colored_data.append(year_data)
df_colored = pd.concat(colored_data)

# Maximalwert für die Y-Achse mit Puffer
max_y = df_colored['TotalPower'].max() * 1.1

import plotly.express as px

# Balkendiagramm mit Animation
fig = px.bar(
    df_colored,
    x='Canton',
    y='TotalPower',
    animation_frame='Year',
    title='Kumulierte PV-Gesamtkapazität pro Kanton (2015–2024)',
    labels={'TotalPower': 'Kumulierte Gesamtkapazität (MW)'},
    color='Color',
    color_discrete_map='identity'
)
fig.update_traces(text=None)
fig.update_layout(
    xaxis=dict(categoryorder='total descending', title='Kanton'),
    yaxis=dict(range=[0, max_y], title='Kumulierte Gesamtkapazität (MW)'),
    template='simple_white',
    showlegend=False
)

# Slider so konfigurieren, dass die Y-Achse dynamisch angepasst wird
fig.layout.sliders[0]['steps'] = [
    dict(
        args=[[str(year)], {"frame": {"duration": 500, "redraw": True}, "mode": "immediate"}],
        label=str(year),
        method="animate"
    )
    for year in df_colored['Year'].unique()
]

fig.write_html("../docs/assets/diagramme/elcom_total_power_2015_2024_kumuliert.html")
fig.show()

## Kumulierte PV-Gesamtkapazität pro Gemeinde (bis 2024)
Die folgende Karte zeigt die kumulierte Gesamtkapazität der Photovoltaik-Anlagen in jeder Schweizer Gemeinde bis zum Jahr 2024.

In [ ]:
import folium
import json
from shapely.geometry import shape, mapping
from shapely.ops import transform
from functools import partial
import pyproj

# CSV mit PLZ und BFS-Nummern laden
file_path = '../data/electricity_production_plants/AMTOVZ_CSV_WGS84.csv'
df_bfs_to_plz = pd.read_csv(file_path, delimiter=';')

# Daten zusammenführen: PV-Anlagen mit BFS-Nummern der Gemeinden
merged_df = df.merge(
    df_bfs_to_plz[['PLZ', 'BFS-Nr']],
    left_on='PostCode',
    right_on='PLZ',
    how='left',
    indicator=True
)

# Kumulative Leistung pro Gemeinde berechnen (bis inkl. 2024)
cumulative_map_data = merged_df[merged_df['Year'] <= 2024].groupby(['BFS-Nr'])['TotalPower'].sum().reset_index()

# GeoJSON der Gemeinden laden
with open('../data/electricity_production_plants/gemeinden.geojson', 'r', encoding='utf-8') as f:
    geojson_data = json.load(f)

# Fehlende Gemeinden mit Durchschnittswert auffüllen
average_power = cumulative_map_data['TotalPower'].mean()
existing_bfs = set(cumulative_map_data['BFS-Nr'])
all_bfs = set(feature['properties']['gemeinde_BFS_NUMMER'] for feature in geojson_data['features'])
missing_bfs = all_bfs - existing_bfs
for bfs in missing_bfs:
    cumulative_map_data = pd.concat([cumulative_map_data, pd.DataFrame({'BFS-Nr': [bfs], 'TotalPower': [average_power]})], ignore_index=True)

# Geometrien vereinfachen und Eigenschaften ergänzen
simplified_features = []
for feature in geojson_data['features']:
    geom = shape(feature['geometry'])
    simplify = partial(
        pyproj.Transformer.from_crs('EPSG:4326', 'EPSG:4326', always_xy=True).transform
    )
    simplified_geom = transform(simplify, geom).simplify(0.001, preserve_topology=True)
    feature['geometry'] = mapping(simplified_geom)
    bfs_nr = feature['properties']['gemeinde_BFS_NUMMER']
    total_power = cumulative_map_data.loc[cumulative_map_data['BFS-Nr'] == bfs_nr, 'TotalPower'].values[0] if bfs_nr in cumulative_map_data['BFS-Nr'].values else 0
    feature['properties']['TotalPower'] = f"{int(total_power):,}".replace(",", "'")
    feature['properties']['bfs-str'] = str(bfs_nr)
    simplified_features.append(feature)
geojson_data['features'] = simplified_features

# Karte initialisieren
map = folium.Map(location=[46.8, 8.33], zoom_start=7)

# Choroplethen-Layer hinzufügen
folium.Choropleth(
    geo_data=geojson_data,
    name='Choropleth',
    data=cumulative_map_data,
    columns=['BFS-Nr', 'TotalPower'],
    key_on='feature.properties.gemeinde_BFS_NUMMER',
    fill_color='YlOrRd',
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='Gesamtkapazität (MW)'
).add_to(map)

# Tooltips und Gemeindegrenzen hinzufügen
folium.GeoJson(
    geojson_data,
    name="Gemeinden",
    style_function=lambda feature: {
        'color': 'black',
        'weight': 0.3,
        'fillOpacity': 0
    },
    tooltip=folium.GeoJsonTooltip(
        fields=['gemeinde_NAME', 'bfs-str', 'TotalPower'],
        aliases=['Gemeinde:', 'BFS-Nummer:', 'Gesamtkapazität (MW):'],
        localize=True
    )
).add_to(map)

# Titel zur Karte hinzufügen
title_html = '''
<div style="font-size:20px;position: absolute;z-index: 1000;left: 15%;"><b>Choroplethenkarte von der Leistung der PV Anlagen pro Gemeinde</b></div>
'''
map.get_root().html.add_child(folium.Element(title_html))

# Karte speichern und anzeigen
map.save('../docs/assets/diagramme/map_cumulative_pv_power.html')
map